In [1]:
#Definindo finais de semana
mes = "08"
lista_fds = ['03','04','10','11','17','18','24','25','31']

# Biblioteca e Funções

In [2]:
####################################-BIBLIOTECAS-####################################

# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#string e data
import unidecode #limpeza
import re #regex

#tempo
import time
from time import sleep

#web scraping
import requests
from bs4 import BeautifulSoup

#automação
from selenium import webdriver #navegador
from selenium.webdriver.common.keys import Keys #comandos do teclado
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait #esperar
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pyautogui #

#manipulação do diretorio
import os

In [3]:
# FUNÇÕES XPATH
def xpath_element(xpath):
    return browser.find_element("xpath", xpath)

def xpath_click(xpath):
    try:
        xpath_element(xpath).click()
    except:
        sleep(2)
        xpath_element(xpath).click()
    
def xpath_send_keys(xpath, texto):
    xpath_element(xpath).send_keys(texto)
    
def xpath_clear(xpath):
    xpath_element(xpath).clear()

def existencia(caminho):
    elementos = browser.find_elements(By.XPATH, caminho)
    return bool(elementos)

In [4]:
#FUNÇÕES
def count_freq(data_frame, coluna):
    freq = data_frame[coluna].value_counts()
    perc = data_frame[coluna].value_counts(normalize = True)*100
    tabela = pd.DataFrame({"Freq":freq, "Perc":perc})
    return display(tabela)

def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        len_antigo = len(dataframe)
        
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe = dataframe.drop_duplicates(subset=['chat'])
        
        print(len(df1), " += ", len_antigo, "=>", len(dataframe))
        
        dataframe.to_excel(nome_arquivo, index=False)
    
    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print(len(dataframe)) 

# ===============================================

# Ligando navegador

In [5]:
#browser.quit(); sleep(5)
browser = webdriver.Firefox()

#fazendo login
browser.get('https://www.pegaplantao.com.br/login')
sleep(1)
xpath_send_keys('//*[@id="MainContent_LoginUser_UserName"]', 'gabriel_s_anjos@yahoo.com') #e-mail
xpath_send_keys('//*[@id="MainContent_LoginUser_Password"]', 'Gabriel2022!') #senha
xpath_click('//*[@id="MainContent_LoginUser_btnLogin"]') #login

In [7]:
browser.get('https://www.pegaplantao.com.br/EscalaMensal/old')

In [8]:
#click setor/profissional
xpath_click('//*[@id="s2id_autogen1"]/a/span[1]')

#click [profissional
xpath_click('//*[@id="select2-drop"]/ul/li[2]/div')

# Capturando setores:

In [10]:
#click criar novo plantão
def criar_plantao():
    xpath_click('//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button')
try:
    criar_plantao()
except:
    sleep(3)

def esperar_elemento_aparecer(elemento, tempo):
    sleep(0.5)
    WebDriverWait(browser, tempo).until(visibility_of(xpath_element(elemento)))

sleep(2)
esperar_elemento_aparecer('//*[@id="service-ckb-list-all-owner"]', 10)
print('apareceu')

def esperar_elemento_desaparecer(elemento, tempo):
    sleep(0.5)
    WebDriverWait(browser, tempo).until_not(visibility_of(xpath_element(elemento)))

apareceu


In [11]:
#CAPTURANDO DADOS DO UNIDADE E SETOR
xpath_click('//*[@id="s2id_ddlServiceSector"]/a/span[1]')
setor_html = xpath_element('//*[@id="select2-drop"]/ul').get_attribute('outerHTML')
soup = BeautifulSoup(setor_html, 'html.parser')

lista_unidade = []
lista_unidade_count = []
lista_plt = []
lista_plt_count = []

count_unidade = 1

for i in soup.find_all(class_="select2-results-dept-0 select2-result select2-result-unselectable select2-result-with-children"):
    #print('\n', i.find(class_='select2-result-label').get_text(), '====== \n')
    count_plt = 1
    for t in i.find_all(class_='select2-results-dept-1 select2-result select2-result-selectable'): 
        #print(t.get_text())
        lista_plt.append(t.get_text())
        lista_plt_count.append(count_plt)
        count_plt += 1

        lista_unidade.append(i.find(class_='select2-result-label').get_text())
        lista_unidade_count.append(count_unidade)
    count_unidade += 1

scrap_unidades = pd.DataFrame({'unidade':lista_unidade, 'momento':lista_plt,
                               'unidade_count':lista_unidade_count, 'plt_count':lista_plt_count})

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: x.replace('  ',' '))

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
scrap_unidades['unidade'] = scrap_unidades['unidade'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
print('OK')

OK


# Funções Selenium

In [12]:
def subir_pg():
    browser.execute_script("window.scrollTo(0, 0);")

In [13]:
def reset_selecao():
    #xpath_click('/html')#; sleep(0.3)
    #xpath_click('/html')

    posicao_antiga = pyautogui.position() 
    pyautogui.click(x=200, y=158); sleep(0.3)
    pyautogui.click(x=200, y=158)
    pyautogui.moveTo(posicao_antiga) 


def fechar_caixa_de_selecao():
    # Localiza um elemento que é seguro para clicar (por exemplo, o corpo da página)
    elemento_seguro = browser.find_element(By.TAG_NAME, 'body')
    # Cria uma ação para clicar no elemento seguro
    ActionChains(browser).move_to_element(elemento_seguro).click().perform()


def reset_selecao1():
    reset_selecao2()
    #fechar_caixa_de_selecao(); sleep(0.3)
    #fechar_caixa_de_selecao()

    posicao_antiga = pyautogui.position() 
    pyautogui.click(x=200, y=158); sleep(0.3)
    pyautogui.click(x=200, y=158)
    pyautogui.moveTo(posicao_antiga)


def reset_selecao2():
    posicao_antiga = pyautogui.position() 
    pyautogui.click(x=200, y=158); sleep(0.3)
    pyautogui.click(x=200, y=158)
    pyautogui.moveTo(posicao_antiga) 
reset_selecao2()

In [14]:
#selecionando setor
def setor(unidade, setor):
    #print('base:', unidade, '-', setor)
    xpath_click('//*[@id="s2id_ddlServiceSector"]/a/span[1]')
    ind = scrap_unidades.loc[(scrap_unidades.unidade == unidade) & 
                             (scrap_unidades.momento == setor)].index.values[0]
    unidade_sel = scrap_unidades['unidade_count'][ind]
    setor_sel = scrap_unidades['plt_count'][ind]
    xpath_click('//*[@id="select2-drop"]/ul/li['+str(unidade_sel)+']/ul/li['+str(setor_sel)+']/div')   
setor("SANTA ISABEL", '04 - SEMANA - CENTRO CIRURGICO')

#inserir e selecionar profissional
def nome_func(nome_profissional):
    #listar todos
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    sleep(0.3)
    xpath_click('//*[@id="s2id_ddlProfessionalA"]/a/span[1]')
    xpath_send_keys('//*[@id="select2-drop"]/div/input', nome_profissional)
    #Dando enter
    xpath_send_keys('//*[@id="select2-drop"]/div/input', Keys.RETURN)
nome_func('OTTO MITTERMAYER')

#inserir horario e duração
def hora(inicio, duracao):
    #Inicio do plantão
    xpath_clear('//*[@id="txtServiceStartTime"]')
    xpath_send_keys('//*[@id="txtServiceStartTime"]', inicio)
    sleep(0.3)
    xpath_clear('//*[@id="txtServiceDuration"]')
    xpath_send_keys('//*[@id="txtServiceDuration"]', duracao)
hora('13:30', '05:00')

In [16]:
#inserir data
def data_func(data_selecionada):
    data_selecionada = str(data_selecionada)
    i_chato2 = str(5) #4 e 5
    i_chato = str(3) #2 e 3

    xpath_clear('/html/body/div['+i_chato2+']/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div['+i_chato+']/input[1]')
    #xpath_clear('/html/body/div[4]/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[3]/input[2]')
    sleep(0.3)
    
    xpath_send_keys('/html/body/div['+i_chato2+']/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div['+i_chato+']/input[1]', data_selecionada)
    #xpath_send_keys('/html/body/div[4]/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[3]/input[2]', data_selecionada)
    reset_selecao2()
data_func("12/03/2024")

In [17]:
def tipo_registro_func(i):
    xpath_click('//*[@id="s2id_add-service-service-types"]/a/span[1]')
    sleep(0.5)

    xpaths = [
        '/html/body/div[12]/div/input',
        '/html/body/div[11]/div/input',
        '/html/body/div[10]/div/input',
        '/html/body/div[9]/div/input',
        '/html/body/div[8]/div/input',
        '/html/body/div[7]/div/input',
        '/html/body/div[6]/div/input',
    ]

    for xpath in xpaths:
        try:
            xpath_send_keys(xpath, i)
            sleep(1)
            xpath_para_clicar = xpath[:xpath.find(']')+1] + '/ul/li/div'
            xpath_click(xpath_para_clicar)
            return  # Sai da função se encontrar um xpath que funcione
        except:
            continue  # Tenta o próximo xpath se ocorrer uma exceção
    
    print('erro')  # Imprime 'erro' se nenhum xpath funcionar

tipo_registro_func('VOLUNTÁRIO')

In [18]:
#inserindo pontuação
def pontuacao(pontos): # NÃO SE ALTERA MAIS
    #xpath_clear('//*[@id="txtServiceValue"]')
    #xpath_send_keys('//*[@id="txtServiceValue"]', pontos) #Pontos
    pass

pontuacao("666")

#selecionar tipo de plantão e ajustando tempo
def tipo_do_registro(duracao, sem_ou_fds, tipo_registro):
    if tipo_registro == 'extra':
        print('EXTRA')
        sleep(0.5)
        reset_selecao2()
        sleep(1)
        tipo_registro_func('VOLUNTÁRIO') #escrevendo
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(200)
            pontuacao(200)
            print('duração >= 5 => 200 pontos')
        else:
            pass
    elif tipo_registro == 'DESLOCADO':
        pass
        '''print('DESLOCADO')
        sleep(1)
        reset_selecao2()
        
        xpath_click('//*[@id="s2id_add-service-service-types"]/a/span[1]')
        xpath_send_keys('/html/body/div[11]/div/input', 'VOLUNTÁRIO') #escrevendo
        xpath_click('/html/body/div[11]/ul/li/div') #clicando no primeiro
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(100)
            pontuacao(100)
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(1)'''
    else:
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(100)
            pontuacao(100)
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(1)

#tipo_do_registro("05:00", "semana", "extra")
#tipo_do_registro(df['DURAÇÃO'][h], df['sem_ou_fds'][h], df['extra'][h])

In [19]:
def insistindo_no_xpath(xpathzinho):
    xpath_click(xpathzinho)
    try:
        esperar_elemento_desaparecer(xpathzinho, 10)
    except:
        xpath_click(xpathzinho)

#########################################################

#salvando plantão
def salvar_plantao():
    xpathzinho = '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]'
    xpath_click(xpathzinho)
#salvar_plantao()

#fechando plantão
def fechar():
    xpathzinho = '//*[@id="modal-add-service"]/div/div[1]/button'
    xpath_click(xpathzinho)
fechar()

#atualizando plantão
xpath_atualizar = '/html/body/div[5]/div/div[1]/div[2]/button[5]'
#xpath_atualizar = '/html/body/div[4]/div/div[1]/div[2]/button[5]'
def atualizando_plantao():
    xpath_click(xpath_atualizar)
#atualizando_plantao()

# Função Tabela

In [44]:
######### ######### FUNÇÃO DE CAPTURA ######### ######### 

#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

#capturando registros
unidades = [
            'ADMINISTRACAO','ALPHA','ANALIA',
            'CENTRAL DO TATUAPE',
            'HOSPITAIS EXTERNOS','HOSPITAL PARA TESTE',
            'ITAIM',
            'JABAQUARA',
            'MATERNIDADE','MORUMBI',
            'OSASCO',
            'SANTA ISABEL',
            'VILLA','VNS'
            ]

def funcao():
    lista_tabela = []
    for i in range(1, 6): #passando por cada semana[1 a 5]
    ##print('semana:', i, "=="*20)
        for t in range(1, 8): #passando por cada dia[1 a 7] dentro da semana
            reg = xpath_element('/html/body/form/main/div[2]/div[2]/table/tbody/tr['+str(i)+']/td['+str(t)+']').text
            content = reg.splitlines() #deixando as palavras em um item dentro de uma lista
            content = [unidecode.unidecode(i.strip().upper()) for i in content] #tratando cada palavra

            dia = content[0] #dia do registro
            del(content[0]) #deletando o dia, ja que foi coletado
            del(content[-1]) #deletando residuo do wepscraping

            if len(content) != 0: #ignorando dias em branco (sem registro) ¹
                #print('dia:', t)

                ##print('dia:', dia)
                #print(content)

                count_hora = 0 #count para capturar a hora de entrada e saida, count=1(entrada) | count=2(saida)
                lista_control = []
                for y in content: #passando um for sobre cada item da lista
                    if y in unidades: # capturando a unidade e dando inicio a novo registro novo
                        #print('================unidade:', y)
                        unidade = y
                    elif y[2] == ':': #reconhecendo se o item é hora (HH*:*mm)
                        count_hora += 1
                        if count_hora == 1: #capturando entrada
                            #print('hora entrada:', y)
                            lista_control.append(y)
                        elif count_hora == 2: #capturando saida
                            #print('hora saida:', y)
                            lista_control.append(y)
                            ##print('lista teste', lista_control)
                            lista_tabela.append(lista_control)
                            lista_control = []
                            count_hora = 0
                    else: #capturando dia, unidade, e periodo
                        #print('.')
                        #print('periodo:', y)
                        lista_control.append(dia)
                        lista_control.append(unidade)
                        lista_control.append(y)
                        lista_control.append(i)
                        lista_control.append(t) #dia semanal

                ##print('='*20)    

            else:
                continue #ignorando dias em branco (sem registro) ²
    
    tabela = pd.DataFrame(lista_tabela, columns = ['Dia','Unidade','Periodo','Semana','Dia Semana','Entrada','Saida'])
    
    #limpeza da tabela
    #########3
    tabela['Dia'] = tabela['Dia'].apply(lambda x: int(x))

    #recolhendo os dias do mes passado que ainda aparecem na primeira semana
    dias_mes_passado = list(tabela[(tabela['Dia'] > 20) & (tabela['Semana'] == 1)].index) 
    tabela.drop(dias_mes_passado, axis=0, inplace=True) #excluindo
    tabela.reset_index(drop=True, inplace=True) #resetando index

    #Inserindo a ordem dos registros
    tabela['ordem'] = ''

    for i in tabela['Dia'].unique():
        for t in range(0, len(tabela[tabela['Dia'] == i].index)):
            tabela['ordem'][tabela[tabela['Dia'] == i].index[t]] = t+1

    #Inserindo coluna de data        
    tabela['data'] = tabela['Dia'].apply(lambda x: str(x))

    for i in range(len(tabela)):
        if len(tabela['data'][i]) == 1:
            tabela['data'][i] = "0" + tabela['data'][i]

        tabela['data'][i] = tabela['data'][i] + "/" + mes + "/2024" 

    tabela = tabela.query("Unidade != 'ADMINISTRACAO'")
    tabela = tabela[~((tabela['Semana'] >= 5) & (tabela['Dia'] < 10))]
    tabela.reset_index(drop=True, inplace=True) #resetando index
    
    tabela['PERIODO'] = np.nan
    for i in range(0, len(tabela)):
        if int(tabela['Entrada'][i][:2]) <= 6:
            tabela['PERIODO'][i] = '6h'
        elif int(tabela['Entrada'][i][:2]) >= 7 and int(tabela['Entrada'][i][:2]) <= 12:
            tabela['PERIODO'][i] = 'manha'
        elif int(tabela['Entrada'][i][:2]) >= 13 and int(tabela['Entrada'][i][:2]) <= 18:
            tabela['PERIODO'][i] = 'tarde'
        else:
            tabela['PERIODO'][i] = 'noturno saida'

    tabela['periodo'] = np.nan
    tabela['periodo'][tabela['PERIODO'] == '6h'] = 1
    tabela['periodo'][tabela['PERIODO'] == 'manha'] = 2
    tabela['periodo'][tabela['PERIODO'] == 'tarde'] = 3
    tabela['periodo'][tabela['PERIODO'] == 'noturno saida'] = 4
    tabela['periodo'][tabela['PERIODO'] == 'noturno entrada'] = 5

    #################################################
    #print(tabela['Periodo'].unique())
    tabela['setor_inicio'] = np.nan
    for i in range(len(tabela)):
        corte = tabela['Periodo'][i].find('-')
        texto = tabela['Periodo'][i][:corte].strip()
        tabela['setor_inicio'][i] = texto
    #print(tabela['setor_inicio'].unique())
    #################################################
    ################################################################################################
    tabela['cor'] = np.nan
    def verificar_cor(dia_sel_filter):
        try:
            endereco = 'tr['+str(dia_sel_filter['Semana'][0])+']/td['+str(dia_sel_filter['Dia Semana'][0])+']/ul['+str(dia_sel_filter['ordem'][0])+']/li'
            xpath = '/html/body/form/main/div[2]/div[2]/table/tbody/'+endereco+'/div[2]/span'
            #print( str(dia_sel['Dia'][0]) )
            cor = xpath_element(xpath).get_attribute('style') #1#
            if cor == 'background-color: rgb(228, 11, 26);':
                return('vermelho')

        except:
            endereco = 'tr['+str(dia_sel_filter['Semana'][0])+']/td['+str(dia_sel_filter['Dia Semana'][0])+']/ul/li['+str(dia_sel_filter['ordem'][0])+']'
            xpath = '/html/body/form/main/div[2]/div[2]/table/tbody/'+endereco+'/div[2]/span'
            #print( str(dia_sel['Dia'][0]) , '----')
            try:
                cor = xpath_element(xpath).get_attribute('style') #2#
                if cor == 'background-color: rgb(228, 11, 26);':
                    return('vermelho')
            except:
                return('prob')
        return(cor)

    for i in range(len(tabela)):
        dia_sel_filter = tabela.loc[i:i+1].reset_index()
        tabela['cor'][i] = verificar_cor(dia_sel_filter)
    ################################################################################################
    tabela = tabela[tabela['Unidade'] != 'HOSPITAIS EXTERNOS'].reset_index(drop=True)

    return tabela
    
tabela = funcao()
tabela.head()

,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo,setor_inicio,cor
0,1,MORUMBI,01 - NOTURNA - ENTRADA ANTES DAS 07HS,1,4,04:30,07:00,1,01/08/2024,6h,1.0,01,"background-color: rgb(2, 184, 252);"
1,1,MORUMBI,04 - SEMANA - CENTRO CIRURGICO,1,4,07:00,18:00,2,01/08/2024,manha,2.0,04,"background-color: rgb(2, 184, 252);"
2,2,MORUMBI,04 - SEMANA - CENTRO CIRURGICO,1,5,07:00,19:00,1,02/08/2024,manha,2.0,04,"background-color: rgb(2, 184, 252);"
3,2,MORUMBI,07 - NOTURNA - SAIDA APOS 19HS,1,5,19:00,19:30,2,02/08/2024,noturno saida,4.0,07,"background-color: rgb(2, 184, 252);"
4,3,MORUMBI,13 - FDS - CENTRO CIRURGICO,1,6,07:00,13:00,1,03/08/2024,manha,2.0,13,"background-color: rgb(92, 184, 162);"


# CAPTURANDO DADOS

## Nomes

In [21]:
def diferenca_horas(horario_inicial, horario_final):
    formato = "%H:%M"
    horario_inicial = datetime.strptime(horario_inicial, formato)
    horario_final = datetime.strptime(horario_final, formato)
    diferenca = horario_final - horario_inicial
    if diferenca.days < 0:
        diferenca = horario_inicial - horario_final
        return diferenca.total_seconds() / 60 * (-1)
    else:
        return diferenca.total_seconds() / 60

In [22]:
# Selecionar nome
def xpath_estranho(i):
    xpaths = [
        '/html/body/div[12]/div/input',
        '/html/body/div[11]/div/input',
        '/html/body/div[10]/div/input',
        '/html/body/div[9]/div/input',
        '/html/body/div[8]/div/input',
        '/html/body/div[7]/div/input',
        '/html/body/div[6]/div/input',
    ]
    
    for xpath in xpaths:
        try:
            xpath_send_keys(xpath, i)
            return  # Sai da função se encontrar um xpath que funcione
        except:
            continue  # Tenta o próximo xpath se ocorrer uma exceção
    
    print('erro')  # Imprime 'erro' se nenhum xpath funcionar
#xpath_estranho(i)

In [23]:
xpath_click('//*[@id="s2id_ddlProfessionals"]/a') #barra de nome
sleep(2)

html = browser.find_element("xpath", '//*[@id="select2-drop"]/ul').get_attribute('outerHTML') #
soup_nome = BeautifulSoup(html, 'html.parser')
#soup_nome
NOMES = []
for i in soup_nome.find_all(class_ ="select2-results-dept-0 select2-result select2-result-selectable"):
    #print(i.get_text())
    NOMES.append(i.get_text())
#del(NOMES[0])

reset_selecao()
display(NOMES)
print(len(NOMES))

['ALESSANDRE LUIZ BRAGA JUNIOR',
 'ALEXANDRE AMARAL LUCAS',
 'ALEXANDRE BERNARDINO',
 'ALEXSANDER TAKASHI HAMADA',
 'ALINE FRANCHINI GOULART LIMA',
 'AMANDA KIYOKO SOUZA KOSSAKA',
 'ANA CLARA MOURÃO BARRETO',
 'ANA GABRIELA GEWEHR VALE SALMEN',
 'ANA JUNG DOCKO',
 'ANDRÉ AUGUSTO BELLINI',
 'ANDRÉ DA COSTA AGUIAR VENTURA',
 'ANDRÉ LUIS XAVIER FRANCO',
 'ANDRÉ MORTARI PLÁ GIL',
 'ANDRÉIA LOUREIRO MORI',
 'ANNA BEATRIZ ARANHA TREVISAN',
 'ANTONIO ALVARO DIAS LIMA DA CUNHA NETO',
 'ARTHUR OLIVEIRA FACCHINI',
 'ARTHUR RODRIGUES TORRELIO',
 'AUGUSTO DE CASTRO SANTOS QUEIROZ',
 'BEATRIZ ANTIPOU DOS SANTOS',
 'BIANCA PRISCILA BUNIOTTI',
 'BRIAN SOSSAI PACHECO',
 'BRUCE YE MAN CHOW',
 'BRUNA DAIBERT DANESIN',
 'BRUNO FERRAZ DE CAMPOS BARONE',
 'BRUNO STORNIOLI TAGUCHI LEMOS',
 'CAIO TULIO DUARTE',
 'CAMILA CHIARANTANO HARRISON',
 'CAMILA RESENDE DE PAULA',
 'CARLOS AUGUSTO TREVISAN',
 'CARLOS EDUARDO CORRÊA GRILO',
 'CARLOS EDUARDO GONDIM OLIVEIRA',
 'CAROLINE FIRMINO LEMES DA SILVA',
 'CLAUDIO

240


# INDO PARA CIMA

In [24]:
count = 0
for NOME in NOMES:
    print(count, '-', NOME)
    count += 1

0 - ALESSANDRE LUIZ BRAGA JUNIOR
1 - ALEXANDRE AMARAL LUCAS
2 - ALEXANDRE BERNARDINO
3 - ALEXSANDER TAKASHI HAMADA
4 - ALINE FRANCHINI GOULART LIMA
5 - AMANDA KIYOKO SOUZA KOSSAKA
6 - ANA CLARA MOURÃO BARRETO
7 - ANA GABRIELA GEWEHR VALE SALMEN
8 - ANA JUNG DOCKO
9 - ANDRÉ AUGUSTO BELLINI
10 - ANDRÉ DA COSTA AGUIAR VENTURA
11 - ANDRÉ LUIS XAVIER FRANCO
12 - ANDRÉ MORTARI PLÁ GIL
13 - ANDRÉIA LOUREIRO MORI
14 - ANNA BEATRIZ ARANHA TREVISAN
15 - ANTONIO ALVARO DIAS LIMA DA CUNHA NETO
16 - ARTHUR OLIVEIRA FACCHINI
17 - ARTHUR RODRIGUES TORRELIO
18 - AUGUSTO DE CASTRO SANTOS QUEIROZ
19 - BEATRIZ ANTIPOU DOS SANTOS
20 - BIANCA PRISCILA BUNIOTTI
21 - BRIAN SOSSAI PACHECO
22 - BRUCE YE MAN CHOW
23 - BRUNA DAIBERT DANESIN
24 - BRUNO FERRAZ DE CAMPOS BARONE
25 - BRUNO STORNIOLI TAGUCHI LEMOS
26 - CAIO TULIO DUARTE
27 - CAMILA CHIARANTANO HARRISON
28 - CAMILA RESENDE DE PAULA
29 - CARLOS AUGUSTO TREVISAN
30 - CARLOS EDUARDO CORRÊA GRILO
31 - CARLOS EDUARDO GONDIM OLIVEIRA
32 - CAROLINE FIRMINO L

In [25]:
'''def apagar_registro(ind):
    #print('PRINCIPAL')
    #abrindo/selecionando periodo conforme localização
    xpath_textao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][ind])+']/td['+str(dia_sel['Dia Semana'][ind])+']/ul['+str(dia_sel['ordem'][ind])+']/li'
    xpath_click(xpath_textao)

    #Esperar componentes carregarem
    WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))

    #Apagando
    esperar_elemento_aparecer('/html/body/div[5]/div/div[1]/div[2]/button[1]', 10)
    xpath_click('/html/body/div[5]/div/div[1]/div[2]/button[1]')
    #Clicando em continuar
    sleep(0.5)
    xpath_click('/html/body/div[5]/div/div[1]/div[3]/button[2]')

    #esperando carregamento do site para prosseguir
    WebDriverWait(browser, 10).until_not(visibility_of(xpath_element('//*[@id="modal-add-service"]/div/div[1]/button')))

#apagar_registro(16)'''

xpath_atualizar = '/html/body/div[5]/div/div[1]/div[2]/button[5]'
def apagando_registro():
    #Apagando
    tipo_registro_func('VOLUNTÁRIO')
    atualizando_plantao()
    try:
        if WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.XPATH, xpath_atualizar))) == False:
            pass
    except:
        atualizando_plantao()
        try: 
            if WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.XPATH, xpath_atualizar))) == False:
                pass
        except:
            print('erro')
#apagando_registro()

def apagar_registro(ind):
    print('')
    '''#abrindo/selecionando periodo conforme localização
    xpath_textao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][ind])+']/td['+str(dia_sel['Dia Semana'][ind])+']/ul['+str(dia_sel['ordem'][ind])+']/li'
    xpath_click(xpath_textao)

    #Esperar componentes carregarem
    WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
    apagando_registro()'''
#apagar_registro()

In [26]:
def verificando_conexoes_de_registros(dia_sel):
    dia_sel_filter = dia_sel.reset_index
    count = 0
    count_controle = 0
    for linha in dia_sel.index:
        if linha < dia_sel.index[-1]:
            if dia_sel['Saida'][linha] == dia_sel['Entrada'][linha+1]:
                pass
            else:
                count_controle =+ 1
    
    if count_controle > 0:
        return(True)
    else:
        return(False)

In [27]:
count_control = 0
lista = []

In [28]:
tempo_espera = 5

In [56]:
reset_selecao()
count_nome = 0

for NOME in NOMES[count_control:]:

    if count_control > 0:
        if NOMES[count_control] != NOMES[count_control-1] or count_nome == 0:
            print('='*20, NOMES[count_control], '='*20)
            count += 1

    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(NOME)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')

    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))
    
    tabela = funcao()
    count = 0
    
    ####################################################################
    #quando o registro for do periodo manhã(2), com saida != 13h => apagar os registros posteriores (tader, noite)
    condicao1 = tabela['periodo'] == 2
    condicao2 = tabela['Saida'] != '13:00'
    condicao3 = tabela['Dia Semana'] < 6
    if len(tabela[condicao1 & condicao2 & condicao3]): 
        for i in tabela[condicao1 & condicao2 & condicao3].index.values:
            if len(tabela[(tabela['data'] == tabela['data'][i]) & (tabela['cor'] != 'vermelho')]) > 0: 
                dia_sel = tabela[(tabela['data'] == tabela['data'][i]) & (tabela['cor'] != 'vermelho')]
                if len(dia_sel[dia_sel['periodo'] > 2]) > 0:
                    if verificando_conexoes_de_registros(dia_sel):
                        print(tabela['data'][i])
                        print('saida manhã != 13h', '\n')
                        sleep(tempo_espera)##
                        for ind in dia_sel[dia_sel['periodo'] > 2].index.values: 
                            apagar_registro(ind)
                        
    ####################################################################
    #quando o registro for do periodo tarde(3), com entrada != 13h => apagar os registros anteriores (manha)
    condicao1 = tabela['periodo'] == 3
    condicao2 = tabela['Entrada'] != '13:00'
    condicao3 = tabela['Dia Semana'] < 6
    if len(tabela[condicao1 & condicao2 & condicao3]):
        for i in tabela[condicao1 & condicao2 & condicao3].index.values:
            if len(tabela[tabela['data'] == tabela['data'][i]]) > 0: 
                dia_sel = tabela[(tabela['data'] == tabela['data'][i]) & (tabela['cor'] != 'vermelho')]
                if len(dia_sel[dia_sel['periodo'] < 3]) > 0:
                    if verificando_conexoes_de_registros(dia_sel):
                        print(tabela['data'][i])
                        print('entrada tarde != 13h', '\n')
                        sleep(tempo_espera)##
                        for ind in dia_sel[dia_sel['periodo'] < 3].index.values: 
                            apagar_registro(ind)

    ##############################
    #quando o registro for do periodo tarde(3), com saida != 19h => apagar os registros posteriores (noite)
    condicao1 = tabela['periodo'] == 3
    condicao2 = tabela['Saida'] != '19:00'
    condicao3 = tabela['Dia Semana'] < 6
    if len(tabela[condicao1 & condicao2 & condicao3]):
        for i in tabela[condicao1 & condicao2 & condicao3].index.values:
            #se o plantão não for o unico no dia, devera ser analisado
            if len(tabela[tabela['data'] == tabela['data'][i]]) > 0: 
                dia_sel = tabela[(tabela['data'] == tabela['data'][i]) & (tabela['cor'] != 'vermelho')]
                #verificando se há plantão antes des deste
                if verificando_conexoes_de_registros(dia_sel):
                    if len(dia_sel[dia_sel['periodo'] > 3]) > 0:
                        print(tabela['data'][i])
                        print('saida tarde != 19h', '\n')
                        sleep(tempo_espera)##
                        ind = dia_sel[dia_sel['periodo'] > 3].index.values[0]
                        apagar_registro(ind)

    ####################################################################
    #quando o registro for do periodo noite(4), com entrada != 19h => apagar os registros anteriores (manha, tarde)
    condicao1 = tabela['periodo'] == 4
    condicao2 = tabela['Entrada'] != '19:00'
    condicao3 = tabela['Dia Semana'] < 6
    if len(tabela[condicao1 & condicao2 & condicao3]):
        for i in tabela[condicao1 & condicao2 & condicao3].index.values:
            if len(tabela[tabela['data'] == tabela['data'][i]]) > 0: 
                dia_sel = tabela[(tabela['data'] == tabela['data'][i]) & (tabela['cor'] != 'vermelho')]
                if len(dia_sel[dia_sel['periodo'] < 4]) > 0:
                    if verificando_conexoes_de_registros(dia_sel):
                        print(tabela['data'][i])
                        print('entrada noite != 19', '\n')
                        sleep(tempo_espera)##
                        count = 0        
                        for ind in dia_sel[dia_sel['periodo'] < 4].index.values: 
                            apagar_registro(ind-count)
                            count += 1
        
    ###############################################################################################################
    #quando os periodos periodos estiverem duplicados => apagar duplicada num 1
    for data in tabela['data'].unique():
        dia_sel = tabela[(tabela['data'] == data) & (tabela['cor'] != 'vermelho')]
        if verificando_conexoes_de_registros(dia_sel):
            if dia_sel['PERIODO'].duplicated().any():
                valores_duplicados = dia_sel['PERIODO'].duplicated()
                indices_duplicados = dia_sel[valores_duplicados].index
                count = 0
                print(data)
                print('plantões duplicados', '\n')
                break
                for i in dia_sel[valores_duplicados]['PERIODO'].values:
                    ind = dia_sel[dia_sel['PERIODO'] == i].index.values[0]
                    print(NOME, tabela['data'][ind], "apagado duplicata")
                    sleep(tempo_espera)##
                    apagar_registro(ind-count)
                    count += 1
                
    ####################################################################
    if count > 0:
        lista.append(count)
        
    count_control += 1
    
print(len(lista))
print(lista)
print(sum(lista))

==================== PEDRO BELINTANI DE MAIO ====================
==================== PEDRO HENRIQUE DINIZ BOTELHO ====================
05/08/2024
saida manhã != 13h 



05/08/2024
plantões duplicados 

==================== PEDRO HENRIQUE LUCAS DE MORAES ====================
26/08/2024
saida manhã != 13h 


==================== PEDRO HENRIQUE MILHOMENS DA MOTA ====================
22/08/2024
saida manhã != 13h 


==================== PEDRO PAULO SILVA FARAH ====================
==================== PEDRO TIAGO BONFITTO ====================
==================== PEDRO VICTOR MASSAROTO E SILVA ====================
20/08/2024
saida manhã != 13h 


==================== RAFAEL QUEIROZ ARAUJO ====================
==================== RAFAEL SOUZA FAVA NERSESSIAN ====================
==================== RAFAEL TADEU MARQUES ====================
==================== RAFHAEL SILVA LEAL ====================
07/08/2024
saida manhã != 13h 


22/08/2024
saida manhã != 13h 


==================== R

KeyboardInterrupt: 

In [ ]:
tabela['data'][41]

In [ ]:
dia_sel = tabela[(tabela['data'] == '31/07/2024') & (tabela['cor'] != 'vermelho')]
dia_sel

237

In [ ]:
count_control += 1

# PREENCHENDO VAZIO

In [34]:
df_total = pd.read_excel('Feitos.xlsx')
df_total.head(2)

,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,...,periodo2,setor,sem_ou_fds,unidade2,setor_inicio,oque_fazer_com_o_registro,controle2,datafeito,hora,chat_pareado
0,[02/08/2024 07:48:38] Alessandre Luiz Braga Ju...,02/08/2024,ALESSANDRE LUIZ BRAGA JUNIOR,02/08 ANALIA - ALESSANDRE ENTRADA 07:00,OK,02/08/2024,ANALIA,NaN,ENTRADA,07:00,...,2.0,04 - SEMANA - CENTRO CIRURGICO,semana,ANALIA,4.0,alterar horario,NaN,20/08/2024,NaN,NaN
1,[05/08/2024 08:35:54] Alessandre Luiz Braga Ju...,05/08/2024,ALESSANDRE LUIZ BRAGA JUNIOR,05/08 JABAQUARA - ALESSANDRE ENTRADA 07:00,OK,05/08/2024,JABAQUARA,NaN,ENTRADA,07:00,...,2.0,04 - SEMANA - CENTRO CIRURGICO,semana,JABAQUARA,4.0,alterar horario,NaN,20/08/2024,NaN,NaN


In [59]:
count_control = 177

In [66]:
reset_selecao()
lista = []
count_nome = 0 

for NOME in NOMES[count_control:]:

    if count_control > 0:
        if NOMES[count_control] != NOMES[count_control-1] or count_nome == 0:
            print('\n', '='*20, NOMES[count_control], '='*20, '\n')
            count += 1
            
    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(NOME)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')

    #esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))
    
    tabela = funcao()
    
    count = 0
    for dia in tabela['Dia'].unique():
        tabela = funcao()
        tabela = tabela[tabela['Dia Semana'] < 6].reset_index(drop=True)
        dia_sel = tabela[tabela['Dia'] == dia].reset_index(drop=True)
        if len(dia_sel['Unidade'].unique()) > 1:
            print('alterando unidades, dia:', dia)

            while len(dia_sel['Unidade'].unique()) > 1:
                
                condicao = df_total['nome'] == unidecode.unidecode(NOME.upper())
                condicao1 = df_total['data'] == dia_sel['data'].values[0]
                unidade_do_dia = df_total['unidade'][condicao & condicao1]
                if len( unidade_do_dia ) == 0:
                    print('VERIFICAR'*5)
                    break
                unidade_do_dia = unidade_do_dia.values[0]
                dia_sel2 = dia_sel[dia_sel['Unidade'] != unidade_do_dia]
                
                print(dia_sel2['Unidade'].values[0], '=>', unidade_do_dia)
                #abrindo/selecionando periodo conforme localização
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel2['Semana'].values[0])+']/td['+str(dia_sel2['Dia Semana'].values[0])+']/ul['+str(dia_sel2['ordem'].values[0])+']/li'
                xpath_click(xpath_localizacao)
                
                #Esperar componentes carregarem
                WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
                sleep(0.5)
                
                #Selecionando Setor
                periodozin = dia_sel2['Periodo'].values[0]
                #periodozin = periodozin.replace('COBERTURA EXTRA','C.CIRURGICO') 
                #periodozin = periodozin.replace('AV.PRE ANESTESICA','C.CIRURGICO')
                try:
                    setor(unidade_do_dia, periodozin)

                    if diferenca_horas(dia_sel2['Entrada'].values[0], dia_sel2['Saida'].values[0]) >= 5*60:
                        pontuacao(100)
                        pontuacao(100)
                        
                    #salvar plantão
                    sleep(1)
                    atualizando_plantao()
                    atualizando_plantao()
                    WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="s2id_ddlProfessionals"]/a/span[1]')))
                    sleep(5)
                except:
                    fechar()

                tabela = funcao()
                tabela = tabela[tabela['Dia Semana'] < 6].reset_index(drop=True)
                dia_sel = tabela[tabela['Dia'] == dia].reset_index(drop=True)
                
    count_control += 1


 ==================== RODRIGO GIOVANELLI CONSTANTINO ==================== 


 ==================== RODRIGO GOMES MINAS NOVAS ==================== 


 ==================== RODRIGO MARTINS PEREIRA ==================== 


 ==================== RODRIGO MIYASHIRO NUNES DOS SANTOS ==================== 


 ==================== ROGÉRIO SIDNEY THON ==================== 


 ==================== RÔMULO SPOLIDORIO ==================== 

alterando unidades, dia: 16
ANALIA => JABAQUARA
alterando unidades, dia: 19
JABAQUARA => ANALIA
alterando unidades, dia: 26
JABAQUARA => ANALIA

 ==================== RONALD FIGUEIREDO GOMES ==================== 


 ==================== SABRINA DE SOUZA RAMOS ==================== 

alterando unidades, dia: 13
ITAIM => JABAQUARA
ITAIM => JABAQUARA

 ==================== SAULLO QUEIROZ SILVEIRA ==================== 


 ==================== SÉRGIO ZAMITH ==================== 


 ==================== SHEILA ITIROCO MONTE ==================== 

alterando unidades, dia:

In [65]:
count_control += 1
print(NOMES[count_control])

RODRIGO GIOVANELLI CONSTANTINO


In [ ]:
condicao1

In [ ]:
/html/body/div[10]/div/input

In [ ]:
dia_sel

In [ ]:
#Selecionando Setor
unidade_do_dia = dia_sel['Unidade'].value_counts().idxmax()

periodozin = dia_sel2['Periodo'].values[0]
periodozin = periodozin.replace('COBERTURA EXTRA','C.CIRURGICO') 
periodozin = periodozin.replace('AV.PRE ANESTESICA','C.CIRURGICO')
setor(unidade_do_dia, periodozin)

if diferenca_horas(dia_sel2['Entrada'].values[0], dia_sel2['Saida'].values[0]) >= 5*60:
    pontuacao(100)
    pontuacao(100)
    
#salvar plantão
sleep(1)
atualizando_plantao()
atualizando_plantao()
sleep(0.2)


count_control += 1

In [ ]:
print(unidade_do_dia)
print(dia_sel['Periodo'][i])

In [ ]:
scrap_unidades['momento'].unique()

In [ ]:
count_control += 1

In [ ]:
fdzão

# MATANDO O RESTO:

In [ ]:
fdzão = fdzão.rename({'Unidade':'unidade', 'Periodo':'setor',
                     'Entrada':'ENTRADA', 'Saida':'SAIDA'}, axis=1)

#DURAÇÃO DO PLANTÃO
fdzão['DURAÇÃO'] = np.nan

for i in range(0, len(fdzão)):
    diferenca_tempo = datetime.strptime(fdzão['SAIDA'][i], '%H:%M') - datetime.strptime(fdzão['ENTRADA'][i], '%H:%M')
    
    #diferenca_tempo vem em formato estranho, tem-se que transformar em segundos antes
    diferenca_tempo_s = diferenca_tempo.seconds
    horas = '0' + str(int((diferenca_tempo_s/60)/60)) #capturando horas
    minutos = str(int((diferenca_tempo_s/60)%60))  #capturando minutos
    
    if len(minutos) == 1:
        minutos = '0' + minutos
    
    fdzão['DURAÇÃO'][i] = horas + ':' + minutos
    
fdzão['DURAÇÃO'].unique()

fd = fdzão[['nome','data','unidade','ENTRADA','SAIDA','DURAÇÃO','setor']]

df = fdzão
df['extra'] = ''

df

In [ ]:
df['sem_ou_fds'] = "semana"

In [ ]:
H=0
print(len(df))

In [ ]:
df = df_prob.reset_index(drop=True)

In [ ]:
for h in range(H, len(df)):
    print(h)
    #click criar novo plantão
    browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button').click()
    
    #esta disponivel o botão de setor
    WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]')))
    #Setor====
    print('base:', df['unidade'][h], '-', df['setor'][h])
    browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').click()
    ind = scrap_unidades.loc[(scrap_unidades['unidade'] == df['unidade'][h]) & 
                             (scrap_unidades['momento'] == df['setor'][h])].index.values[0]
    unidade = scrap_unidades['unidade_count'][ind]
    setorzin = scrap_unidades['plt_count'][ind]
    print('scraping:', scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
    #SELECIONANDO Setor ##//*[@id="select2-drop"]/ul/li[unidade]/ul/li[momento]/div
    print('caixa:', browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/div').text, '-',
          browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').text)
    browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').click()
    print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').text, '\n')
    
    #PROFISSIONAL
    #esta disponivel o botão de profissional
    WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]')))
    
    #listar todos
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click()
    
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
    sleep(0.5)
    browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').click()
    browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(df['nome'][h])
    #Dando enter
    browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(Keys.RETURN)
    #info
    print('base:', df['nome'][h])
    print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').text)
    print(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').text)
    
    sleep(2)
    #Data
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').clear()
    sleep(1)
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').send_keys(df['data'][h])
    sleep(0.5)
    print(df['data'][h])
    
    #Inicio do plantão
    browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').clear()
    browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').send_keys(df['ENTRADA'][h])
    sleep(1)
    print('hora de entrada:', df['ENTRADA'][h], '\nhora de saida:', df['SAIDA'][h])
    #Duração do plantão
    browser.find_element("xpath", '//*[@id="txtServiceDuration"]').clear()
    browser.find_element("xpath", '//*[@id="txtServiceDuration"]').send_keys(df['DURAÇÃO'][h])
    print('duração do plantão', df['DURAÇÃO'][h])
    sleep(2)
    
    ##TIPO
    '''
    //*[@id="select2-drop"]/ul/li[3]/div
    1 - Normal
    2 - DESLOCAMENTO
    3 - VOLUNTÁRIO
    4 - COBERTURA
    '''
    if df['extra'][h] == 'extra':
        print('EXTRA')
        sleep(1)
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        sleep(1)
        #selecionando o tipo voluntario:
        browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[3]/div').click()
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(200) #Pontos
            print('duração >5 -> 200 pontos')
        else:
            pass
    elif df['extra'][h] == 'DESLOCADO':
        print('DESLOCADO')
        sleep(1)
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        sleep(1)
        #selecionando o tipo voluntario:
        browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[2]/div').click()
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(100) #Pontos
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(2)
    else:
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(100) #Pontos
            print('duração >5 -> 100 pontos')
            
            sleep(1)
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(100) #Pontos
        else:
            pass
        sleep(2)
        
    #salvar plantão
    sleep(1)
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    sleep(4)

    #VERIFICANDO SE HOUVE ERRO - HÁ PLANTÃO NO MESMO HORARIO
    if browser.find_element("xpath", '/html/body/div[5]/div/div[2]/div[2]/div/label').text != '':
        #browser.find_element("xpath", '//*[@id="ckb-add-service-ignore-conflict"]').click()#estou ciente
        sleep(1)
        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click() #salvar plantão
        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        #
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #FECHAR
        df.loc[int(h), 'controle'] = "PROBS"
        print("PROB")
    else:
        df.loc[int(h), 'controle'] = "NORM"
    
    #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #fechar {X}

    #não esta disponivel o X
    
    WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button')))
    H += 1
    print('='*40, '\n')
    
    if H % 5 == 0:
        print('='*80)
        print(df['controle'].value_counts())
        print('='*80)
        
print("="*50, 'TERMINOU', "="*50)
print('Total  ', len(df))
print(df['controle'].value_counts())

In [ ]:
H += 1

In [ ]:
#df_prob = pd.concat([df_prob, df[df['controle'] != 'NORM']]).reset_index(drop=True)
df_prob = df[df['controle'] != 'NORM'].reset_index(drop=True)
print(len(df_prob))
df = df[df['controle'] == 'NORM'].reset_index(drop=True)
print(len(df))

In [ ]:
def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe.to_excel(nome_arquivo, index=False)
        print("+=", len(df1), '; len =', len(dataframe)) 

    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print("len =", len(dataframe)) 
        
salvando_nome_arquivo('Feitos_a+.xlsx', df)
print('.')
salvando_nome_arquivo('Não Feitos.xlsx', df_prob)

In [ ]:
df_prob

In [ ]:
browser.quit()